# Part A: Subreddit Prediction ##

### Preparation

In [0]:
subreddit_train = "coursework_subreddit_train.json"
subreddit_test = "coursework_subreddit_test.json"

!gsutil cp gs://textasdata/coursework/coursework_subreddit_train.json $subreddit_train 
!gsutil cp gs://textasdata/coursework/coursework_subreddit_test.json  $subreddit_test

Copying gs://textasdata/coursework/coursework_subreddit_train.json...
/ [1 files][ 10.1 MiB/ 10.1 MiB]                                                
Operation completed over 1 objects/10.1 MiB.                                     
Copying gs://textasdata/coursework/coursework_subreddit_test.json...
/ [1 files][  2.7 MiB/  2.7 MiB]                                                
Operation completed over 1 objects/2.7 MiB.                                      


In [0]:
import pandas as pd

train_threads = pd.read_json(path_or_buf=subreddit_train, lines=True)
print(list(train_threads.columns.values))
print(train_threads.head())
print(train_threads.size)

['is_self_post', 'posts', 'subreddit', 'title', 'url']
   is_self_post                                              posts  \
0           1.0  [{'body': 'I think everyone has that one frien...   
1           1.0  [{'body': 'I not 100% sure this is the right p...   
2           1.0  [{'body': '', 'author': 'Leisure321', 'url': '...   
3           1.0  [{'body': 'It's called 'forgetting things'.', ...   
4           1.0  [{'body': 'How would I do this? I am looking t...   

        subreddit                                              title  \
0   relationships  How do I [23F] communicate with my self-center...   
1  summonerschool  What Cherry switch do you recommend for League...   
2       askreddit                   Where do memes go when they die?   
3           trees                     Some weird long term affects??   
4        buildapc  Simple question: If I install Windows to a sta...   

                                                 url  
0  https://www.reddit.com/r/relation

In [0]:
test_threads = pd.read_json(path_or_buf=subreddit_test, lines=True)
print(test_threads.head())
print(test_threads.size)

   is_self_post                                              posts  \
0           1.0  [{'body': 'Was watching a VOD from last years ...   
1           1.0  [{'body': 'Basically what the title says.', 'u...   
2           1.0  [{'body': '', 'author': 'Daft-Punk', 'url': 'h...   
3           1.0  [{'body': 'I start running this year. I do it ...   
4           1.0  [{'body': '[deleted]', 'url': 'https://www.red...   

       subreddit                                              title  \
0      starcraft  Just a reminder on how much SC2 has evolved th...   
1    whowouldwin  Your Favorite Hero Now Has A Healing Factor As...   
2      askreddit  If you could live anywhere in the world, where...   
3      askreddit                   Do you ever get use to exercise?   
4  tipofmytongue         [TOMT] [book] A scary french book for kids   

                                                 url  
0  https://www.reddit.com/r/starcraft/comments/mq...  
1  https://www.reddit.com/r/whowouldwin/co

In [0]:
subreddit_counts = train_threads['subreddit'].value_counts()
print(subreddit_counts.describe())
top_subbreddits = subreddit_counts.nlargest(20)
top_subbreddits_list = top_subbreddits.index.tolist()
print(top_subbreddits)

count     20.000000
mean      72.800000
std       73.368285
min       28.000000
25%       36.250000
50%       45.500000
75%       63.750000
max      334.000000
Name: subreddit, dtype: float64
askreddit               334
leagueoflegends         196
buildapc                131
explainlikeimfive        82
trees                    66
techsupport              63
gaming                   62
pcmasterrace             62
electronic_cigarette     59
relationships            48
tipofmytongue            43
hearthstone              38
jailbreak                38
summonerschool           37
atheism                  37
reddit.com               34
whowouldwin              33
movies                   33
personalfinance          32
starcraft                28
Name: subreddit, dtype: int64


### Q1

In [0]:
#construct train data dataframe
import json
train_temp = list()

with open(subreddit_train) as jsonfile:
  for i, line in enumerate(jsonfile):
    thread = json.loads(line)
    posts = ""
    authors = ""
    # Keep information about the source thread where the post comes from.
    for post in thread['posts']:
      posts+=post.get('body', "") + " "
      authors+=post.get('author', "") + " "
      len_of_posts = str(len(posts))
      len_of_thread = str(len(thread['subreddit']))
    
    train_temp.append((thread['subreddit'], 
                    thread['title'], 
                    authors,
                    posts,
                    len_of_posts,
                    len_of_thread))
print(len(train_temp))

labels = ['subreddit', 'title',  'author', 'body', 'len_of_posts', 'len_of_thread']
train_data = pd.DataFrame(train_temp, columns=labels)
train_data

1456


,subreddit,title,author,body,len_of_posts,len_of_thread
0,relationships,How do I [23F] communicate with my self-center...,Pouritdownmythroat WhyFrankWhy Pouritdownmythr...,I think everyone has that one friend who loves...,2518,13
1,summonerschool,What Cherry switch do you recommend for League...,ThisGermanGuy shaunrnm Sub_Salac MisterBlack...,I not 100% sure this is the right place to pos...,3358,14
2,askreddit,Where do memes go when they die?,Leisure321 Zeolance Buttersgoo23 quzimaa Esca...,Facebook 9gag Memes never truly die Dickbutt...,136,9
3,trees,Some weird long term affects??,refugee4chan donquixote6179 ThatStonedAsianGuy,It's called 'forgetting things'. Dude i dont t...,291,5
4,buildapc,Simple question: If I install Windows to a sta...,catalyzeme uses rvabdn greg2709 spydr101,How would I do this? I am looking to wait anot...,1449,8
5,whowouldwin,Sentry vs. Swamp Thing,gpacman21 jeansplice Ame-no-nobuko vadergeek A...,[Respect Sentry.](http://www.reddit.com/r/resp...,3214,11
6,summonerschool,Hows zed going this season?,ellosmello fozzix VarusMidOrAfk I_know_where_y...,Is he still a good pick? I played him alot in ...,1025,14
7,trees,Favorite hip-hop/r&b to listen to while high?,walkngonawire ResonantCascade snicklepants jui...,"We all know about stoner rock, stoner metal, s...",1847,5
8,personalfinance,Opening an IRA as a college student,patrick6h aBoglehead thisismyfirstpost twblalock,"Hi, I am looking to open an IRA while I still ...",1025,15
9,leagueoflegends,LCS spots secured for corporations or players?!,CrazyChunckMaster420 xmodusterz Frogad Waheyy ...,I am just shocked by how Riot decided to handl...,1334,15


In [0]:
#construct train data dataframe
import json
test_temp = list()

with open(subreddit_test) as jsonfile:
  for i, line in enumerate(jsonfile):
    thread = json.loads(line)
    posts = ""
    authors = ""
    # Keep information about the source thread where the post comes from.
    for post in thread['posts']:
      posts+=post.get('body', "") + " "
      authors+=post.get('author', "") + " "
      len_of_posts = str(len(posts))
      len_of_thread = str(len(thread['subreddit']))
    
    test_temp.append((thread['subreddit'], 
                    thread['title'], 
                    authors,
                    posts,
                    len_of_posts,
                    len_of_thread))
print(len(test_temp))

labels = ['subreddit', 'title',  'author', 'body', 'len_of_posts', 'len_of_thread']
test_data = pd.DataFrame(test_temp, columns=labels)
test_data

365


,subreddit,title,author,body,len_of_posts,len_of_thread
0,starcraft,Just a reminder on how much SC2 has evolved th...,MeisterKarl smsy vehemus NruJaC,Was watching a VOD from last years DreamHack W...,764,9
1,whowouldwin,Your Favorite Hero Now Has A Healing Factor As...,TheAnti-Monitor iamcatch22 blames_irrationall...,Basically what the title says. Danny Phantom i...,2846,11
2,askreddit,"If you could live anywhere in the world, where...",Daft-Punk AutoModerator Memithezombiekiller I...,On a beach someplace warm and far away from a...,1609,9
3,askreddit,Do you ever get use to exercise?,theawe_some DrWallyHayes IGrammarGood theawe_s...,I start running this year. I do it frequently ...,1884,9
4,tipofmytongue,[TOMT] [book] A scary french book for kids,grapesandmilk,[deleted] Could you possibly provide us a clue...,481,13
5,trees,I Don't Understand Why People Hate the War On ...,duel_dude Knotfloyd KuronekoKawaii vishnukg ra...,I think they make pretty good music: https://w...,847,5
6,askreddit,What do you miss the most from childhood?,woodstock01 pernoe Coutinho_Fan logallama beck...,Being carefree and having someone else worry ...,700,9
7,tipofmytongue,[TOMT] [Song] I heard this on the radio,gunthug gunthug cpaige88,I recorded my voice http://www.speakpipe.com/v...,283,13
8,explainlikeimfive,ELI5: Why is pee warm?,throwawaymayblol pnotar throwawaymayblol paulk...,It should come out at the same temperature as...,311,17
9,hearthstone,Undocumented change: Stall prevention,Orconem Hjelpen wpScraps,"So I meant to post this yesterday, but in one ...",461,11


In [0]:
#construct labels
train_labels = train_data['subreddit']
test_labels = test_data['subreddit']

In [0]:
import spacy

# Load the medium english model. 
!python -m spacy download en_core_web_md

nlp = spacy.load('en_core_web_md', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')

# Download a stopword list
import nltk
nltk.download('stopwords')


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#Tokenize
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    normalized = token.text.lower().strip()
    if ((token.is_alpha or token.is_digit)):
      normalized_tokens.append(normalized)
  return normalized_tokens
  return normalized_tokens

#Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))

In [0]:
# Evaluation: accuracy, macro-averaged precision, recall, and F1 measures

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

def evaluation_summary(description, predictions, true_labels):
  print("Evaluation for: " + description)
  precision = precision_score(predictions, true_labels,average='macro')
  recall = recall_score(predictions, true_labels, average='macro')
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1, average='macro') 
  print("Classifier '%s' has Acc=%0.3f P=%0.3f R=%0.3f F1=%0.3f" % (description,accuracy,precision,recall,f1))
  print(classification_report(predictions, true_labels, digits=3))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [0]:
# Select feature
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.    """

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

### vectorization

In [0]:
# one-hot encoding vectorizor
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_1 = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('one-hot', CountVectorizer(tokenizer=tokenize_normalize, binary=True)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('one-hot', CountVectorizer(tokenizer=tokenize_normalize, binary=True)), 
              ])),
            ('author', Pipeline([
              ('selector', ItemSelector(key='author')),
              ('one-hot', CountVectorizer(tokenizer=spacy_tokenize, binary=True)), 
              ])),
        ])
        )
    ])

In [0]:
# TF-IDF vectorizor
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_2 = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('author', Pipeline([
              ('selector', ItemSelector(key='author')),
              ('tf-idf', TfidfVectorizer(tokenizer=spacy_tokenize)), 
              ])),
        ])
        )
    ])

In [0]:
# Construct train/test features using onr-hot encoding vectorizor
one_hot_train_features = prediction_pipeline_1.fit_transform(train_data)
one_hot_test_features = prediction_pipeline_1.transform(test_data)

In [0]:
# Construct train/test features using TF-IDF vectorizor
tf_idf_train_features = prediction_pipeline_2.fit_transform(train_data)
tf_idf_test_features = prediction_pipeline_2.transform(test_data)

### DummyClassifier

In [0]:
#DummyClassifier, one-hot encoding
from sklearn.dummy import DummyClassifier

dummy_prior = DummyClassifier(strategy='stratified')
dummy_prior.fit(one_hot_train_features, train_labels)
print(dummy_prior.score(one_hot_test_features, test_labels))
evaluation_summary("Dummy Prior with one-hot", dummy_prior.predict(one_hot_test_features), test_labels)

dummy_mf = DummyClassifier(strategy='most_frequent')
dummy_mf.fit(one_hot_train_features, train_labels)
print(dummy_mf.score(one_hot_test_features, test_labels))
evaluation_summary("Dummy Majority with one-hot", dummy_mf.predict(one_hot_test_features), test_labels)


DummyClassifier(constant=None, random_state=None, strategy='stratified')

0.1232876712328767
Evaluation for: Dummy Prior with one-hot
Classifier 'Dummy Prior with one-hot' has Acc=0.107 P=0.051 R=0.055 F1=0.052
                      precision    recall  f1-score   support

           askreddit      0.250     0.263     0.256        80
             atheism      0.000     0.000     0.000         5
            buildapc      0.108     0.108     0.108        37
electronic_cigarette      0.000     0.000     0.000        17
   explainlikeimfive      0.071     0.053     0.061        19
              gaming      0.059     0.091     0.071        11
         hearthstone      0.133     0.200     0.160        10
           jailbreak      0.000     0.000     0.000        10
     leagueoflegends      0.125     0.098     0.110        61
              movies      0.000     0.000     0.000         7
        pcmasterrace      0.087     0.118     0.100        17
     personalfinance      0.000     0.000     0.000         3
          reddit.com      0.000     0.000     0.000     

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

0.23013698630136986
Evaluation for: Dummy Majority with one-hot
Classifier 'Dummy Majority with one-hot' has Acc=0.230 P=0.050 R=0.012 F1=0.019
                      precision    recall  f1-score   support

           askreddit      1.000     0.230     0.374       365
             atheism      0.000     0.000     0.000         0
            buildapc      0.000     0.000     0.000         0
electronic_cigarette      0.000     0.000     0.000         0
   explainlikeimfive      0.000     0.000     0.000         0
              gaming      0.000     0.000     0.000         0
         hearthstone      0.000     0.000     0.000         0
           jailbreak      0.000     0.000     0.000         0
     leagueoflegends      0.000     0.000     0.000         0
              movies      0.000     0.000     0.000         0
        pcmasterrace      0.000     0.000     0.000         0
     personalfinance      0.000     0.000     0.000         0
          reddit.com      0.000     0.000     0.0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
#DummyClassifier, TF-IDF
from sklearn.dummy import DummyClassifier

dummy_prior = DummyClassifier(strategy='stratified')
dummy_prior.fit(tf_idf_train_features, train_labels)
print(dummy_prior.score(tf_idf_test_features, test_labels))
evaluation_summary("Dummy Prior with TF-IDF", dummy_prior.predict(tf_idf_test_features), test_labels)

dummy_mf = DummyClassifier(strategy='most_frequent')
dummy_mf.fit(tf_idf_train_features, train_labels)
print(dummy_mf.score(tf_idf_test_features, test_labels))
evaluation_summary("Dummy Majority with TF-IDF", dummy_mf.predict(tf_idf_test_features), test_labels)


DummyClassifier(constant=None, random_state=None, strategy='stratified')

0.10136986301369863
Evaluation for: Dummy Prior with TF-IDF
Classifier 'Dummy Prior with TF-IDF' has Acc=0.107 P=0.039 R=0.048 F1=0.043
                      precision    recall  f1-score   support

           askreddit      0.310     0.317     0.313        82
             atheism      0.000     0.000     0.000         9
            buildapc      0.081     0.091     0.086        33
electronic_cigarette      0.000     0.000     0.000        12
   explainlikeimfive      0.000     0.000     0.000        27
              gaming      0.000     0.000     0.000        23
         hearthstone      0.067     0.125     0.087         8
           jailbreak      0.091     0.100     0.095        10
     leagueoflegends      0.104     0.111     0.108        45
              movies      0.000     0.000     0.000        11
        pcmasterrace      0.087     0.167     0.114        12
     personalfinance      0.000     0.000     0.000         8
          reddit.com      0.000     0.000     0.000      

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

0.23013698630136986
Evaluation for: Dummy Majority with TF-IDF
Classifier 'Dummy Majority with TF-IDF' has Acc=0.230 P=0.050 R=0.012 F1=0.019
                      precision    recall  f1-score   support

           askreddit      1.000     0.230     0.374       365
             atheism      0.000     0.000     0.000         0
            buildapc      0.000     0.000     0.000         0
electronic_cigarette      0.000     0.000     0.000         0
   explainlikeimfive      0.000     0.000     0.000         0
              gaming      0.000     0.000     0.000         0
         hearthstone      0.000     0.000     0.000         0
           jailbreak      0.000     0.000     0.000         0
     leagueoflegends      0.000     0.000     0.000         0
              movies      0.000     0.000     0.000         0
        pcmasterrace      0.000     0.000     0.000         0
     personalfinance      0.000     0.000     0.000         0
          reddit.com      0.000     0.000     0.000

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### LogisticRegression

In [0]:
#Classifier 1: LogisticRegression, one-hot encoding
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr_model = lr.fit(one_hot_train_features, train_labels)
print(lr_model.score(one_hot_test_features, test_labels))
evaluation_summary("LogisticRegression with one-hot", lr_model.predict(one_hot_test_features), test_labels)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6931506849315069
Evaluation for: LogisticRegression with one-hot
Classifier 'LogisticRegression with one-hot' has Acc=0.693 P=0.589 R=0.775 F1=0.623
                      precision    recall  f1-score   support

           askreddit      0.929     0.678     0.784       115
             atheism      0.500     0.857     0.632         7
            buildapc      0.757     0.800     0.778        35
electronic_cigarette      0.889     0.800     0.842        10
   explainlikeimfive      0.786     0.611     0.688        18
              gaming      0.471     0.421     0.444        19
         hearthstone      0.467     1.000     0.636         7
           jailbreak      0.545     0.857     0.667         7
     leagueoflegends      0.896     0.566     0.694        76
              movies      0.400     1.000     0.571         2
        pcmasterrace      0.217     0.625     0.323         8
     personalfinance      0.900     1.000     0.947         9
          reddit.com      0.000     0.000 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
#Classifier 1: LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr_model = lr.fit(tf_idf_train_features, train_labels)
print(lr_model.score(tf_idf_test_features, test_labels))
evaluation_summary("LogisticRegression  with TF-IDF", lr_model.predict(tf_idf_test_features), test_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.547945205479452
Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.548 P=0.354 R=0.637 F1=0.380
                      precision    recall  f1-score   support

           askreddit      0.952     0.440     0.602       182
             atheism      0.083     1.000     0.154         1
            buildapc      0.838     0.775     0.805        40
electronic_cigarette      0.667     0.750     0.706         8
   explainlikeimfive      0.357     0.714     0.476         7
              gaming      0.059     0.250     0.095         4
         hearthstone      0.067     1.000     0.125         1
           jailbreak      0.182     1.000     0.308         2
     leagueoflegends      0.854     0.488     0.621        84
              movies      0.000     0.000     0.000         0
        pcmasterrace      0.087     0.667     0.154         3
     personalfinance      0.500     1.000     0.667         5
          reddit.com      0.000     0.000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


###  SVC Classifier

In [0]:
#Classifier 2: SVC Classifier, one-hot encoding
from sklearn.svm import SVC
svc_calssifier = SVC()
svc_model = svc_calssifier.fit(one_hot_train_features, train_labels) 
print(svc_model.score(one_hot_test_features, test_labels))
evaluation_summary("SVC Classifier with one-hot", svc_model.predict(one_hot_test_features), test_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.23013698630136986
Evaluation for: SVC Classifier with one-hot
Classifier 'SVC Classifier with one-hot' has Acc=0.230 P=0.050 R=0.012 F1=0.019
                      precision    recall  f1-score   support

           askreddit      1.000     0.230     0.374       365
             atheism      0.000     0.000     0.000         0
            buildapc      0.000     0.000     0.000         0
electronic_cigarette      0.000     0.000     0.000         0
   explainlikeimfive      0.000     0.000     0.000         0
              gaming      0.000     0.000     0.000         0
         hearthstone      0.000     0.000     0.000         0
           jailbreak      0.000     0.000     0.000         0
     leagueoflegends      0.000     0.000     0.000         0
              movies      0.000     0.000     0.000         0
        pcmasterrace      0.000     0.000     0.000         0
     personalfinance      0.000     0.000     0.000         0
          reddit.com      0.000     0.000     0.0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
#Classifier 2: SVC Classifier, TF-IDF
from sklearn.svm import SVC
svc_calssifier = SVC()
svc_model = svc_calssifier.fit(tf_idf_train_features, train_labels) 
print(svc_model.score(tf_idf_test_features, test_labels))
evaluation_summary("SVC Classifier  with TF-IDF", svc_model.predict(tf_idf_test_features), test_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.23013698630136986
Evaluation for: SVC Classifier  with TF-IDF
Classifier 'SVC Classifier  with TF-IDF' has Acc=0.230 P=0.050 R=0.012 F1=0.019
                      precision    recall  f1-score   support

           askreddit      1.000     0.230     0.374       365
             atheism      0.000     0.000     0.000         0
            buildapc      0.000     0.000     0.000         0
electronic_cigarette      0.000     0.000     0.000         0
   explainlikeimfive      0.000     0.000     0.000         0
              gaming      0.000     0.000     0.000         0
         hearthstone      0.000     0.000     0.000         0
           jailbreak      0.000     0.000     0.000         0
     leagueoflegends      0.000     0.000     0.000         0
              movies      0.000     0.000     0.000         0
        pcmasterrace      0.000     0.000     0.000         0
     personalfinance      0.000     0.000     0.000         0
          reddit.com      0.000     0.000     0.0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### BernoulliNB

In [0]:
#Classifier 3: BernoulliNB, one-hot encoding
from sklearn.naive_bayes import BernoulliNB
classifier = BernoulliNB()
nb_model = classifier.fit(one_hot_train_features, train_labels)
print(nb_model.score(one_hot_test_features, test_labels))
evaluation_summary("BernoulliNB with one-hot", nb_model.predict(one_hot_test_features), test_labels)


0.3013698630136986
Evaluation for: BernoulliNB with one-hot
Classifier 'BernoulliNB with one-hot' has Acc=0.301 P=0.083 R=0.072 F1=0.062
                      precision    recall  f1-score   support

           askreddit      1.000     0.262     0.415       321
             atheism      0.000     0.000     0.000         0
            buildapc      0.541     0.800     0.645        25
electronic_cigarette      0.000     0.000     0.000         0
   explainlikeimfive      0.000     0.000     0.000         2
              gaming      0.000     0.000     0.000         0
         hearthstone      0.000     0.000     0.000         0
           jailbreak      0.000     0.000     0.000         0
     leagueoflegends      0.125     0.375     0.188        16
              movies      0.000     0.000     0.000         0
        pcmasterrace      0.000     0.000     0.000         0
     personalfinance      0.000     0.000     0.000         0
          reddit.com      0.000     0.000     0.000     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
#Classifier 3: BernoulliNB, TF-IDF
from sklearn.naive_bayes import BernoulliNB
classifier = BernoulliNB()
nb_model = classifier.fit(tf_idf_train_features, train_labels)
print(nb_model.score(tf_idf_test_features, test_labels))
evaluation_summary("BernoulliNB  with TF-IDF", nb_model.predict(tf_idf_test_features), test_labels)


0.3013698630136986
Evaluation for: BernoulliNB  with TF-IDF
Classifier 'BernoulliNB  with TF-IDF' has Acc=0.301 P=0.083 R=0.072 F1=0.062
                      precision    recall  f1-score   support

           askreddit      1.000     0.262     0.415       321
             atheism      0.000     0.000     0.000         0
            buildapc      0.541     0.800     0.645        25
electronic_cigarette      0.000     0.000     0.000         0
   explainlikeimfive      0.000     0.000     0.000         2
              gaming      0.000     0.000     0.000         0
         hearthstone      0.000     0.000     0.000         0
           jailbreak      0.000     0.000     0.000         0
     leagueoflegends      0.125     0.375     0.188        16
              movies      0.000     0.000     0.000         0
        pcmasterrace      0.000     0.000     0.000         0
     personalfinance      0.000     0.000     0.000         0
          reddit.com      0.000     0.000     0.000     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Q2

In [0]:
# LogisticRegression with TF-IDF vectorizor 
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.    """

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
      
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_3 = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)),            
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('author', Pipeline([
              ('selector', ItemSelector(key='author')),
              ('tf-idf', TfidfVectorizer(tokenizer=spacy_tokenize)), 
              ])),
          ])),
         ('logreg', LogisticRegression())
    ])

### parameters tuning

In [0]:
parameter_sublinear_tf = {'union__body__tf-idf__sublinear_tf':(True, False)}
parameter_ngram_range = {'union__body__tf-idf__ngram_range':((1,1), (1,2), (1,3))}
parameter_max_features = {'union__body__tf-idf__max_features':(None, 5000, 10000, 50000)}

parameter_C = {'logreg__C':[1000, 10000]}
parameter_multi_class = {'logreg__multi_class':('ovr', 'multinomial', 'auto')}
parameter_solver =  {'logreg__solver':('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}

In [0]:
# sublinear_tf
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(prediction_pipeline_3, param_grid=parameter_sublinear_tf, n_jobs=1, verbose=1, scoring='f1_macro', cv=2)
print("Performing grid search...")
print("pipeline:", [name for name, _ in prediction_pipeline_3.steps])
print("parameters:")
print(parameter_sublinear_tf)
# grid_search.get_params().keys()
grid_search.fit(train_data, train_labels)

# best choice: False
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameter_sublinear_tf.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['union', 'logreg']
parameters:
{'union__body__tf-idf__sublinear_tf': (True, False)}
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   43.1s finished


Best score: 0.353
Best parameters set:
	union__body__tf-idf__sublinear_tf: False


In [0]:
# ngram_range
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(prediction_pipeline_3, param_grid=parameter_ngram_range, n_jobs=1, verbose=1, scoring='f1_macro', cv=2)
print("Performing grid search...")
print("pipeline:", [name for name, _ in prediction_pipeline_3.steps])
print("parameters:")
print(parameter_ngram_range)
#grid_search.get_params().keys()
grid_search.fit(train_data, train_labels)
# best choice: (1, 1)
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameter_ngram_range.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['union', 'logreg']
parameters:
{'union__body__tf-idf__ngram_range': ((1, 1), (1, 2), (1, 3))}
Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.8min finished


Best score: 0.353
Best parameters set:
	union__body__tf-idf__ngram_range: (1, 1)


In [0]:
# max_features 
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(prediction_pipeline_3, param_grid=parameter_max_features, n_jobs=1, verbose=1, scoring='f1_macro', cv=2)
print("Performing grid search...")
print("pipeline:", [name for name, _ in prediction_pipeline_3.steps])
print("parameters:")
print(parameter_max_features)
grid_search.fit(train_data, train_labels)
# best choice: 5000
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameter_max_features.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['union', 'logreg']
parameters:
{'union__body__tf-idf__max_features': (None, 5000, 10000, 50000)}
Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.4min finished


Best score: 0.355
Best parameters set:
	union__body__tf-idf__max_features: 5000


In [0]:
# parameter_C
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(prediction_pipeline_3, param_grid=parameter_C, n_jobs=1, verbose=1, scoring='f1_macro', cv=2)
print("Performing grid search...")
print("pipeline:", [name for name, _ in prediction_pipeline_3.steps])
print("parameters:")
print(parameter_C)
grid_search.fit(train_data, train_labels)
# best choice: 10000
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameter_C.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['selector', 'one-hot', 'logreg']
parameters:
{'logreg__C': [1000, 10000]}
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   46.5s finished


Best score: 0.395
Best parameters set:
	logreg__C: 10000


In [0]:
# parameter_solver
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(prediction_pipeline_3, param_grid=parameter_solver, n_jobs=1, verbose=1, scoring='f1_macro', cv=2)
print("Performing grid search...")
print("pipeline:", [name for name, _ in prediction_pipeline_3.steps])
print("parameters:")
print(parameter_solver)
grid_search.fit(train_data, train_labels)
# best choice: liblinear
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameter_solver.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['union', 'logreg']
parameters:
{'logreg__solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}
Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "th

Best score: 0.353
Best parameters set:
	logreg__solver: 'liblinear'


In [0]:
# parameter_multi_class
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(prediction_pipeline_3, param_grid=parameter_multi_class, n_jobs=1, verbose=1, scoring='f1_macro', cv=2)
print("Performing grid search...")
print("pipeline:", [name for name, _ in prediction_pipeline_3.steps])
print("parameters:")
print(parameter_multi_class)
grid_search.fit(train_data, train_labels)
# best choice: auto
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameter_multi_class.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['union', 'logreg']
parameters:
{'logreg__multi_class': ('ovr', 'multinomial', 'auto')}
Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.5min finished


Best score: 0.353
Best parameters set:
	logreg__multi_class: 'auto'


### error analysis

In [0]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_4 = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize, max_features = 5000)),            
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize, max_features = 5000)), 
              ])),
            ('author', Pipeline([
              ('selector', ItemSelector(key='author')),
              ('tf-idf', TfidfVectorizer(tokenizer=spacy_tokenize, max_features = 5000)), 
              ])),
          ])),
         ('logreg', LogisticRegression(C=10000, solver='saga', multi_class='auto'))
    ])

In [0]:
# LogisticRegression,TF-IDF, with tuned parameters
prediction_pipeline_4.fit(train_data, train_labels)
evaluation_summary("LogisticRegression,TF-IDF, with tuned parameters", prediction_pipeline_4.predict(test_data), test_labels)                   

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Evaluation for: LogisticRegression,TF-IDF, with tuned parameters
Classifier 'LogisticRegression,TF-IDF, with tuned parameters' has Acc=0.710 P=0.608 R=0.738 F1=0.623
                      precision    recall  f1-score   support

           askreddit      0.869     0.652     0.745       112
             atheism      0.417     0.833     0.556         6
            buildapc      0.865     0.842     0.853        38
electronic_cigarette      0.889     0.727     0.800        11
   explainlikeimfive      1.000     0.636     0.778        22
              gaming      0.471     0.471     0.471        17
         hearthstone      0.667     1.000     0.800        10
           jailbreak      0.727     1.000     0.842         8
     leagueoflegends      0.854     0.719     0.781        57
              movies      0.200     0.500     0.286         2
        pcmasterrace      0.304     0.778     0.438         9
     personalfinance      0.800     0.889     0.842         9
          reddit.com      0

In [0]:
predictions = pd.Series(prediction_pipeline_4.predict(test_data)) 
comp = test_data
comp = comp.assign(predict=predictions.values)
comp[['subreddit', 'predict']]

,subreddit,predict
0,starcraft,gaming
1,whowouldwin,whowouldwin
2,askreddit,askreddit
3,askreddit,askreddit
4,tipofmytongue,tipofmytongue
5,trees,askreddit
6,askreddit,askreddit
7,tipofmytongue,tipofmytongue
8,explainlikeimfive,explainlikeimfive
9,hearthstone,leagueoflegends


### feature development

In [0]:
#Tokenize
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    normalized = token.text.lower().strip()
    if ((token.is_alpha or token.is_digit or !token.is_stop)): #new feature 1: stopwords handling
      normalized_tokens.append(normalized)
  return normalized_tokens
  return normalized_tokens

#Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))

In [0]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_5 = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize, max_features = 5000)),            
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize, max_features = 5000)), 
              ])),
            ('author', Pipeline([
              ('selector', ItemSelector(key='author')),
              ('tf-idf', TfidfVectorizer(tokenizer=spacy_tokenize, max_features = 5000)), 
              ])),
            ('len_of_posts', Pipeline([
              ('selector', ItemSelector(key='len_of_posts')),
              ('tf-idf', TfidfVectorizer(tokenizer=spacy_tokenize, max_features = 5000)),     #new feature 2: length of posts       
              ])),
#             ('len_of_thread', Pipeline([
#               ('selector', ItemSelector(key='len_of_posts')),
#               ('tf-idf', TfidfVectorizer(tokenizer=spacy_tokenize, max_features = 5000)),            
#               ])),
          ])),
         ('logreg', LogisticRegression(C=10000, solver='saga', multi_class='auto'))
    ])

In [0]:
# LogisticRegression,TF-IDF, with added features
prediction_pipeline_5.fit(train_data, train_labels)
evaluation_summary("LogisticRegression,TF-IDF, with added features", prediction_pipeline_5.predict(test_data), test_labels)         

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('body', Pipeline(memory=None,
     steps=[('selector', ItemSelector(key='body')), ('tf-idf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='cont...penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False))])

Evaluation for: LogisticRegression,TF-IDF, with added features
Classifier 'LogisticRegression,TF-IDF, with added features' has Acc=0.658 P=0.523 R=0.676 F1=0.544
                      precision    recall  f1-score   support

           askreddit      0.917     0.570     0.703       135
             atheism      0.333     1.000     0.500         4
            buildapc      0.865     0.744     0.800        43
electronic_cigarette      0.778     0.700     0.737        10
   explainlikeimfive      0.786     0.611     0.688        18
              gaming      0.235     0.400     0.296        10
         hearthstone      0.467     1.000     0.636         7
           jailbreak      0.727     1.000     0.842         8
     leagueoflegends      0.875     0.646     0.743        65
              movies      0.200     0.500     0.286         2
        pcmasterrace      0.174     0.667     0.276         6
     personalfinance      0.700     0.875     0.778         8
          reddit.com      0.000

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Part B: Discourse prediction ##

### Q3

In [0]:
discourse_train = "coursework_discourse_train.json"
discourse_test = "coursework_discourse_test.json"
  
!gsutil cp gs://textasdata/coursework/coursework_discourse_train.json $discourse_train  
!gsutil cp gs://textasdata/coursework/coursework_discourse_test.json  $discourse_test

Copying gs://textasdata/coursework/coursework_discourse_train.json...
- [1 files][ 60.2 MiB/ 60.2 MiB]                                                
Operation completed over 1 objects/60.2 MiB.                                     
Copying gs://textasdata/coursework/coursework_discourse_test.json...
/ [1 files][ 15.1 MiB/ 15.1 MiB]                                                
Operation completed over 1 objects/15.1 MiB.                                     


In [0]:
# The reddit thread structure is nested with posts in a new content.
# This block reads the file as json and creates a new data frame.
import pandas as pd
import json

def load_posts(file):
  # A temporary variable to store the list of post content.
  posts_tmp = list()

  with open(file) as jsonfile:
    for i, line in enumerate(jsonfile):
     # if (i > 2): break
      thread = json.loads(line)
      for post in thread['posts']:
        # NOTE: This could be changed to use additional features from the post or thread.
        # DO NOT change the labels for the test set.
        posts_tmp.append((thread['subreddit'], thread['title'], thread['url'],
                        post['id'], post.get('author', ""), post.get('body', ""), post.get("majority_link", ""), 
                        post.get('post_depth', 0), post.get('majority_type', ""), # discourse type label 
                        post.get('in_reply_to', "") ))

# Create the posts data frame.  
  labels = ['subreddit', 'title', 'url', 'id', 'author', 'body', 'majority_link', 
          'post_depth', 'discourse_type', 'in_reply_to']
  return pd.DataFrame(posts_tmp, columns=labels)

In [0]:
train_posts = load_posts(discourse_train)
# Filter out empty labels
train_posts = train_posts[train_posts['discourse_type'] != ""]
print(train_posts.head())
print("Num posts: ", train_posts.size)

    subreddit                           title  \
0  worldofpvp  Help me decide my new PvP main   
1  worldofpvp  Help me decide my new PvP main   
2  worldofpvp  Help me decide my new PvP main   
3  worldofpvp  Help me decide my new PvP main   
4  worldofpvp  Help me decide my new PvP main   

                                                 url          id  \
0  https://www.reddit.com/r/worldofpvp/comments/2...   t3_2v0anq   
1  https://www.reddit.com/r/worldofpvp/comments/2...  t1_codb2p9   
2  https://www.reddit.com/r/worldofpvp/comments/2...  t1_codg0we   
3  https://www.reddit.com/r/worldofpvp/comments/2...  t1_coeatsq   
4  https://www.reddit.com/r/worldofpvp/comments/2...  t1_codbyit   

         author                                               body  \
0      TyrickEU  Hi. \nAs a raider previously, i had no problem...   
1          vurt  [deleted]  \n ^^^^^^^^^^^^^^^^0.5422 \n > [Wha...   
2   OptimusNice  This goes mostly for 3v3 since that seems to b...   
3               

The label for the post we will be predicting is in the discourse_type column.

In [0]:
test_posts = load_posts(discourse_test)
# Filter out empty labels
test_posts = test_posts[test_posts['discourse_type'] != ""]
print("Num posts: ", test_posts.size)


Num posts:  198120


In [0]:
train_labels = train_posts['discourse_type']
test_labels = test_posts['discourse_type']

Examine the distribution over labels on the training data.

In [0]:
discourse_counts = train_labels.value_counts()
print(discourse_counts.describe())

top_discourse = discourse_counts.nlargest(200)
print(top_discourse)
top_discourse = top_discourse.index.tolist()
print(top_discourse)

count       10.000000
mean      7926.700000
std       9664.321866
min       1266.000000
25%       1671.500000
50%       3235.500000
75%      11919.750000
max      31419.000000
Name: discourse_type, dtype: float64
answer              31419
elaboration         14775
question            13610
appreciation         6849
agreement            3868
disagreement         2603
humor                1787
other                1633
announcement         1457
negativereaction     1266
Name: discourse_type, dtype: int64
['answer', 'elaboration', 'question', 'appreciation', 'agreement', 'disagreement', 'humor', 'other', 'announcement', 'negativereaction']


In [0]:
import spacy

# Load the medium english model. 
!python -m spacy download en_core_web_md

nlp = spacy.load('en_core_web_md', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')

#Tokenize
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    normalized = token.text.lower().strip()
    if ((token.is_alpha or token.is_digit)):
      normalized_tokens.append(normalized)
  return normalized_tokens
  return normalized_tokens

#Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))

    100% |████████████████████████████████| 120.9MB 96.7MB/s 
  Running setup.py install for en-core-web-md ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [0]:
# Evaluation: accuracy, macro-averaged precision, recall, and F1 measures

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

def evaluation_summary(description, predictions, true_labels):
  print("Evaluation for: " + description)
  precision = precision_score(predictions, true_labels,average='macro')
  recall = recall_score(predictions, true_labels, average='macro')
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1, average='macro') 
  print("Classifier '%s' has Acc=%0.3f P=%0.3f R=%0.3f F1=%0.3f" % (description,accuracy,precision,recall,f1))
  print(classification_report(predictions, true_labels, digits=3))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [0]:
# Select feature
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.    """

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
      
# TF-IDF vectorizor
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_a = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize, max_features = 5000)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize, max_features = 5000)), 
              ])),
        ])
        )
    ])

# Define the features
X_train = prediction_pipeline_a.fit_transform(train_posts)
X_test = prediction_pipeline_a.transform(test_posts)

from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='stratified',random_state=0)
clf.fit(X_train, train_posts['discourse_type'])
predictions = clf.predict(X_test)  
evaluation_summary("DummyClassifier with TF-IDF", predictions, test_posts['discourse_type'])

Evaluation for: DummyClassifier with TF-IDF
Classifier 'DummyClassifier with TF-IDF' has Acc=0.233 P=0.101 R=0.101 F1=0.101
                  precision    recall  f1-score   support

       agreement      0.055     0.056     0.055       934
    announcement      0.022     0.021     0.021       382
          answer      0.393     0.395     0.394      7910
    appreciation      0.079     0.085     0.082      1603
    disagreement      0.037     0.035     0.036       686
     elaboration      0.187     0.183     0.185      3711
           humor      0.024     0.024     0.024       449
negativereaction      0.026     0.027     0.026       297
           other      0.021     0.020     0.021       395
        question      0.167     0.166     0.167      3445

       micro avg      0.233     0.233     0.233     19812
       macro avg      0.101     0.101     0.101     19812
    weighted avg      0.233     0.233     0.233     19812


Confusion matrix:
 [[  52   20  379   66   38  165   28   19

In [0]:
# LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10000, solver='saga', multi_class='auto')
logreg_model = logreg.fit(X_train, train_posts['discourse_type'])
predictions = logreg_model.predict(X_test)  
evaluation_summary("LogisticRegression  with TF-IDF", predictions, test_posts['discourse_type'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.399 P=0.243 R=0.239 F1=0.240
                  precision    recall  f1-score   support

       agreement      0.232     0.201     0.215      1102
    announcement      0.115     0.066     0.084       639
          answer      0.544     0.528     0.536      8173
    appreciation      0.525     0.517     0.521      1745
    disagreement      0.090     0.092     0.091       630
     elaboration      0.226     0.266     0.244      3074
           humor      0.081     0.097     0.088       383
negativereaction      0.091     0.103     0.097       271
           other      0.106     0.091     0.098       438
        question      0.419     0.427     0.423      3357

       micro avg      0.399     0.399     0.399     19812
       macro avg      0.243     0.239     0.240     19812
    weighted avg      0.401     0.399     0.399     19812


Confusion matrix:
 [[ 221   30  313   73   43  141  

In [0]:
predict = pd.Series(predictions)
test_tmp = test_posts
test_tmp = test_tmp.assign(predict=predict.values)
test_tmp

,subreddit,title,url,id,author,body,majority_link,post_depth,discourse_type,in_reply_to,predict
0,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t3_1ds5ds,sobeisforlovers,Edit: I'm in the Oklahoma City and Tulsa Oklah...,none,0,question,,announcement
1,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9tbz9b,KevinAndEarth,are you in the USA? i would say that unless t...,t3_1ds5ds,1,question,t3_1ds5ds,disagreement
2,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9tcqh8,,[deleted],t1_c9tbz9b,2,answer,t1_c9tbz9b,announcement
3,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9thky3,EnglishTraitor,Great info! It shows that stopping on a highwa...,t1_c9tcqh8,3,appreciation,t1_c9tcqh8,answer
4,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9twaqh,pixelmonger,"Lately, it's been my experience that taking ne...",,2,elaboration,t1_c9tbz9b,question
5,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9tg275,RKIvey,"Speaking as a LEO, if you're not blocking traf...",t3_1ds5ds,1,answer,t3_1ds5ds,announcement
6,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9tbwts,m1ss1ontomars2k4,"I'd assume it is, given that it's supposed to ...",t3_1ds5ds,1,answer,t3_1ds5ds,answer
7,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9tgn0x,projecthouse,"At least in the US, highway is a very broad te...",t3_1ds5ds,1,answer,t3_1ds5ds,answer
8,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9thtlv,texasphotog,I would be worried more about the danger assoc...,t3_1ds5ds,1,answer,t3_1ds5ds,answer
9,photography,Is it against the law to stop on the side of a...,https://www.reddit.com/r/photography/comments/...,t1_c9tboi2,Maxion,"Yes, and no and in some places there probably ...",t3_1ds5ds,1,answer,t3_1ds5ds,answer


### Q4

### preparation

In [0]:
discourse_train = "coursework_discourse_train.json"
discourse_test = "coursework_discourse_test.json"
  
!gsutil cp gs://textasdata/coursework/coursework_discourse_train.json $discourse_train  
!gsutil cp gs://textasdata/coursework/coursework_discourse_test.json  $discourse_test

Copying gs://textasdata/coursework/coursework_discourse_train.json...
- [1 files][ 60.2 MiB/ 60.2 MiB]                                                
Operation completed over 1 objects/60.2 MiB.                                     
Copying gs://textasdata/coursework/coursework_discourse_test.json...
- [1 files][ 15.1 MiB/ 15.1 MiB]                                                
Operation completed over 1 objects/15.1 MiB.                                     


In [0]:
import pandas as pd
import json

def load_posts(file):
  # A temporary variable to store the list of post content.
  posts_tmp = list()

  with open(file) as jsonfile:
    for i, line in enumerate(jsonfile):
     # if (i > 2): break
      thread = json.loads(line)
      initial_author = ""
      num_of_com = 0
      for post in thread['posts']:
        num_of_com+=1
        if(post.get('in_reply_to', "") == ""):
          initial_author = post.get('author', "")
      for post in thread['posts']:
        if(post.get('author', "") == initial_author):
          is_initial_author = True
        else:
          is_initial_author = False
        
        posts_tmp.append((thread['subreddit'], 
                          str(post.get('post_depth', 0)),
                          str(num_of_com),
                          str(is_initial_author),
                          thread['title'],
                          post.get('body', ""),
                          
                          thread['url'],
                        post['id'], post.get('author', ""),  post.get("majority_link", ""), 
                         post.get('majority_type', ""), # discourse type label 
                        post.get('in_reply_to', "") ))

# Create the posts data frame.  
  labels = ['subreddit', 'post_depth', 'num_of_com', 'is_initial_author',
            
            'title', 'body', 'url', 'id', 'author', 'majority_link', 
           'discourse_type', 'in_reply_to']
  return pd.DataFrame(posts_tmp, columns=labels)

In [0]:
# Load the medium english model. 
import spacy
!python -m spacy download en_core_web_md

nlp = spacy.load('en_core_web_md', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')

#Tokenize
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    normalized = token.text.lower().strip()
    if ((token.is_alpha or token.is_digit)):
      normalized_tokens.append(normalized)
  return normalized_tokens
  return normalized_tokens

#Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))

# Select feature
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.    """

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [0]:
train_posts = load_posts(discourse_train)
# Filter out empty labels
train_posts = train_posts[train_posts['discourse_type'] != ""]

test_posts = load_posts(discourse_test)
# Filter out empty labels
test_posts = test_posts[test_posts['discourse_type'] != ""]

train_labels = train_posts['discourse_type']
test_labels = test_posts['discourse_type']

In [0]:
# Evaluation: accuracy, macro-averaged precision, recall, and F1 measures

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

def evaluation_summary(description, predictions, true_labels):
  print("Evaluation for: " + description)
  precision = precision_score(predictions, true_labels,average='macro')
  recall = recall_score(predictions, true_labels, average='macro')
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1, average='macro') 
  print("Classifier '%s' has Acc=%0.3f P=%0.3f R=%0.3f F1=%0.3f" % (description,accuracy,precision,recall,f1))
  print(classification_report(predictions, true_labels, digits=3))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

### Feature 1

In [0]:
# Additional feature 1: subreddit
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_a = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('subreddit', Pipeline([
              ('selector', ItemSelector(key='subreddit')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
        ])
        )
    ])

# Define the features
X_train = prediction_pipeline_a.fit_transform(train_posts)
X_test = prediction_pipeline_a.transform(test_posts)

In [0]:
# LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10000, solver='saga', multi_class='auto')
logreg_model = logreg.fit(X_train, train_posts['discourse_type'])
predictions = logreg_model.predict(X_test)  
evaluation_summary("LogisticRegression  with TF-IDF", predictions, test_posts['discourse_type'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.417 P=0.242 R=0.283 F1=0.253
                  precision    recall  f1-score   support

       agreement      0.216     0.284     0.245       723
    announcement      0.115     0.126     0.120       334
          answer      0.562     0.519     0.540      8608
    appreciation      0.538     0.607     0.570      1523
    disagreement      0.064     0.126     0.085       326
     elaboration      0.267     0.266     0.267      3642
           humor      0.050     0.139     0.074       165
negativereaction      0.088     0.231     0.127       117
           other      0.074     0.163     0.102       172
        question      0.449     0.366     0.403      4202

       micro avg      0.417     0.417     0.417     19812
       macro avg      0.242     0.283     0.253     19812
    weighted avg      0.442     0.417     0.428     19812


Confusion matrix:
 [[ 205   11  340   58   16  168  

### Feature 2

In [0]:
# Additional feature 2: post_depth
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_a = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('post_depth', Pipeline([
              ('selector', ItemSelector(key='post_depth')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
        ])
        )
    ])

# Define the features
X_train = prediction_pipeline_a.fit_transform(train_posts)
X_test = prediction_pipeline_a.transform(test_posts)

In [0]:
# LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10000, solver='saga', multi_class='auto')
logreg_model = logreg.fit(X_train, train_posts['discourse_type'])
predictions = logreg_model.predict(X_test)  
evaluation_summary("LogisticRegression  with TF-IDF", predictions, test_posts['discourse_type'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.534 P=0.335 R=0.374 F1=0.348
                  precision    recall  f1-score   support

       agreement      0.274     0.331     0.300       788
    announcement      0.510     0.581     0.543       320
          answer      0.708     0.634     0.669      8871
    appreciation      0.548     0.605     0.575      1558
    disagreement      0.117     0.198     0.147       379
     elaboration      0.393     0.373     0.383      3817
           humor      0.053     0.125     0.074       192
negativereaction      0.088     0.199     0.122       136
           other      0.082     0.136     0.103       228
        question      0.578     0.562     0.570      3523

       micro avg      0.534     0.534     0.534     19812
       macro avg      0.335     0.374     0.348     19812
    weighted avg      0.562     0.534     0.546     19812


Confusion matrix:
 [[ 261    0  262   77   36  220  

### Feature 3

In [0]:
# Additional Feature 3: num_of_com
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_a = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('num_of_com', Pipeline([
              ('selector', ItemSelector(key='num_of_com')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
        ])
        )
    ])

# Define the features
X_train = prediction_pipeline_a.fit_transform(train_posts)
X_test = prediction_pipeline_a.transform(test_posts)

In [0]:
# LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10000, solver='saga', multi_class='auto')
logreg_model = logreg.fit(X_train, train_posts['discourse_type'])
predictions = logreg_model.predict(X_test)  
evaluation_summary("LogisticRegression  with TF-IDF", predictions, test_posts['discourse_type'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.412 P=0.244 R=0.281 F1=0.255
                  precision    recall  f1-score   support

       agreement      0.224     0.282     0.250       755
    announcement      0.107     0.137     0.120       285
          answer      0.540     0.515     0.527      8320
    appreciation      0.531     0.582     0.555      1568
    disagreement      0.078     0.146     0.102       343
     elaboration      0.304     0.268     0.285      4123
           humor      0.050     0.141     0.074       163
negativereaction      0.094     0.199     0.128       146
           other      0.085     0.160     0.111       200
        question      0.429     0.376     0.401      3909

       micro avg      0.412     0.412     0.412     19812
       macro avg      0.244     0.281     0.255     19812
    weighted avg      0.430     0.412     0.419     19812


Confusion matrix:
 [[ 213   15  339   61   19  165  

### Feature 4

In [0]:
# Additional feature 4: is_initial_author
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_a = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)),
              ])),
           
            ('is_initial_author', Pipeline([
              ('selector', ItemSelector(key='is_initial_author')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            
        ])
        )
    ])

# Define the features
X_train = prediction_pipeline_a.fit_transform(train_posts)
X_test = prediction_pipeline_a.transform(test_posts)

In [0]:
# LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10000, solver='saga', multi_class='auto')
logreg_model = logreg.fit(X_train, train_posts['discourse_type'])
predictions = logreg_model.predict(X_test)  
evaluation_summary("LogisticRegression  with TF-IDF", predictions, test_posts['discourse_type'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.445 P=0.265 R=0.299 F1=0.275
                  precision    recall  f1-score   support

       agreement      0.223     0.304     0.257       698
    announcement      0.247     0.265     0.256       339
          answer      0.597     0.543     0.569      8735
    appreciation      0.534     0.596     0.563      1543
    disagreement      0.069     0.141     0.092       313
     elaboration      0.279     0.270     0.274      3747
           humor      0.033     0.089     0.048       169
negativereaction      0.085     0.198     0.119       131
           other      0.077     0.146     0.101       198
        question      0.504     0.438     0.468      3939

       micro avg      0.445     0.445     0.445     19812
       macro avg      0.265     0.299     0.275     19812
    weighted avg      0.473     0.445     0.457     19812


Confusion matrix:
 [[ 212   12  380   57   19  153  

### Feature 5

In [0]:
# Additional feature 5: character ngram
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_a = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('body2', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize, ngram_range = (1,4))), 
              ])),
           
        ])
        )
    ])

# Define the features
X_train = prediction_pipeline_a.fit_transform(train_posts)
X_test = prediction_pipeline_a.transform(test_posts)

In [0]:
# LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10000, solver='saga', multi_class='auto')
logreg_model = logreg.fit(X_train, train_posts['discourse_type'])
predictions = logreg_model.predict(X_test)  
evaluation_summary("LogisticRegression  with TF-IDF", predictions, test_posts['discourse_type'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.503 P=0.291 R=0.363 F1=0.302
                  precision    recall  f1-score   support

       agreement      0.246     0.441     0.316       531
    announcement      0.178     0.125     0.147       521
          answer      0.694     0.551     0.614      9993
    appreciation      0.599     0.706     0.648      1459
    disagreement      0.066     0.235     0.102       179
     elaboration      0.180     0.329     0.233      1986
           humor      0.050     0.232     0.083        99
negativereaction      0.094     0.299     0.144        97
           other      0.125     0.224     0.160       210
        question      0.682     0.493     0.572      4737

       micro avg      0.503     0.503     0.503     19812
       macro avg      0.291     0.363     0.302     19812
    weighted avg      0.589     0.503     0.534     19812


Confusion matrix:
 [[ 234   16  407   52    5   97  

### Feature 6

In [0]:
# Use the tokenizer to extract all tokens from the body of the posts.
# Flatten the tokens in the post into a single list of all the tokens.
!pip install --upgrade gensim
import gensim
import itertools
all_tokens = []
all_posts_tokenized = train_posts.body.apply(tokenize_normalize)
all_tokens = list(itertools.chain.from_iterable(all_posts_tokenized))

model = gensim.models.Word2Vec(all_posts_tokenized, size=50, window=5, \
                               min_count=5, sg=0, alpha=0.025, iter=10, batch_words=10000)

    100% |████████████████████████████████| 24.2MB 1.7MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class AverageEmbeddingVectorizer(BaseEstimator, TransformerMixin
                                ):
    def __init__(self, embedding_model):
        self.embedding = embedding_model
        self.dimension = embedding_model.vector_size

    def fit(self, X, y):
        return self
      
    def transform(self, X):  
      # Skip OOV terms. Return 0 if no words are in the vocabulary.
      #print (X)
      return np.array([ 
          np.mean([self.embedding[token] for token in doc if token in self.embedding]
                or [np.zeros(self.dimension)], axis=0)
          for doc in X
      ])
    
    
w2v_vectorizer = AverageEmbeddingVectorizer(model)

In [0]:
def create_w2v(x):
  doc = tokenize_normalize(x)
  X = [doc]
  w2c_vector = w2v_vectorizer.transform(X)
  string = ""
  for v in w2c_vector[0]:
    string += str(v) + " "
  return string
  
train_posts['w2v'] = train_posts.body.apply(create_w2v)
test_posts['w2v'] = test_posts.body.apply(create_w2v)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
# Additional feature 6: word2vector
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_a = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('w2v', Pipeline([
              ('selector', ItemSelector(key='w2v')),
              ('tf-idf', TfidfVectorizer(tokenizer=spacy_tokenize)), 
              ])), 	
        ])
        )
    ])

# Define the features
X_train = prediction_pipeline_a.fit_transform(train_posts)
X_test = prediction_pipeline_a.transform(test_posts)

In [0]:
# LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg_model = logreg.fit(X_train, train_posts['discourse_type'])
predictions = logreg_model.predict(X_test)  
evaluation_summary("LogisticRegression  with TF-IDF", predictions, test_posts['discourse_type'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.509 P=0.235 R=0.505 F1=0.248
                  precision    recall  f1-score   support

       agreement      0.183     0.563     0.276       309
    announcement      0.008     1.000     0.016         3
          answer      0.830     0.510     0.632     12919
    appreciation      0.587     0.764     0.664      1321
    disagreement      0.017     0.282     0.032        39
     elaboration      0.185     0.304     0.230      2217
           humor      0.007     0.214     0.013        14
negativereaction      0.026     0.471     0.049        17
           other      0.037     0.400     0.068        35
        question      0.466     0.543     0.501      2938

       micro avg      0.509     0.509     0.509     19812
       macro avg      0.235     0.505     0.248     19812
    weighted avg      0.673     0.509     0.561     19812


Confusion matrix:
 [[ 174    0  509   33    2  139  

### Features combined

In [0]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Use FeatureUnion to combine the features from post body, thread title and post author id
prediction_pipeline_a = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('is_initial_author', Pipeline([
              ('selector', ItemSelector(key='is_initial_author')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])), 
            ('num_of_com', Pipeline([
              ('selector', ItemSelector(key='num_of_com')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])), 	
            ('subreddit', Pipeline([
              ('selector', ItemSelector(key='subreddit')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])), 	
            ('post_depth', Pipeline([
              ('selector', ItemSelector(key='post_depth')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])), 	
            ('body2', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf-idf', TfidfVectorizer(tokenizer=tokenize_normalize, ngram_range = (1,4))), 
              ])),
            ('w2v', Pipeline([
              ('selector', ItemSelector(key='w2v')),
              ('tf-idf', TfidfVectorizer(tokenizer=spacy_tokenize)), 
              ])), 	
             
        ])
        )
    ])

# Define the features
X_train = prediction_pipeline_a.fit_transform(train_posts)
X_test = prediction_pipeline_a.transform(test_posts)

In [0]:
# LogisticRegression,TF-IDF
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg_model = logreg.fit(X_train, train_posts['discourse_type'])
predictions = logreg_model.predict(X_test)  
evaluation_summary("LogisticRegression  with TF-IDF", predictions, test_posts['discourse_type'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Evaluation for: LogisticRegression  with TF-IDF
Classifier 'LogisticRegression  with TF-IDF' has Acc=0.624 P=0.324 R=0.554 F1=0.353
                  precision    recall  f1-score   support

       agreement      0.279     0.567     0.374       467
    announcement      0.189     0.734     0.301        94
          answer      0.868     0.652     0.744     10576
    appreciation      0.601     0.778     0.678      1327
    disagreement      0.062     0.354     0.106       113
     elaboration      0.496     0.433     0.463      4162
           humor      0.026     0.364     0.049        33
negativereaction      0.029     0.429     0.055        21
           other      0.037     0.483     0.069        29
        question      0.649     0.744     0.693      2990

       micro avg      0.624     0.624     0.624     19812
       macro avg      0.324     0.554     0.353     19812
    weighted avg      0.714     0.624     0.656     19812


Confusion matrix:
 [[ 265    0  274   41    3  322  